<a href="https://colab.research.google.com/github/azuzow/oogaBooga/blob/master/OogaBooga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#all libraries used
!apt install chromium-chromedriver
!pip install selenium 
!pip install fuzzywuzzy
import math
import random
import urllib.request
import requests
import ssl
import time
from googlesearch import search
import re
from bs4 import BeautifulSoup
!pip install --upgrade firebase-admin
from fuzzywuzzy import fuzz
from selenium import webdriver
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (79.0.3945.79-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.
Requirement already up-to-date: firebase-admin in /usr/local/lib/python3.6/dist-packages (3.2.1)


In [0]:
#gets all webpages for query
pagesWithoutRankings=[]
amzonLinks=[]
def getLinks(query,linkcount=40):
  domainList = []
  searchList= []
  for count,i in enumerate (search(query, tld="com", num=linkcount, start=0, stop=linkcount, pause=2)):
      domain = i.split('//')[-1].split('/')[0]
      domain = domain.lstrip('www.')
      domain = "https://www." + domain
      #if a domain has already been seen dont add the link to the pages that will be scraped
      if domain not in domainList:
          domainList.append(domain)
          print(count)
          # print(i)
          searchList.append(i)
  # print(searchList)
  return searchList
def visible(element):
  if element.parent.name in ['style', 'script', '[document]', 'head', 'title']:
    return False
  elif re.match('<!--.*-->', str(element.encode('utf-8'))):
    return False
  return True

def getPages(urls):
  pages = []
  cc=1
  amazonLinks=set()
  #problems occur here, i think when try doesnt work the first time? im not sure
  junkWords=["These ","quite","to","too","like","This",'this','because',"it's"]
  # junkWords=[]

  for items in urls:
      try:
          html = requests.get(items,timeout=5).text
          soup = BeautifulSoup(html,'html.parser')

          data = soup.findAll(text=True)

          #finds all links then gets all links with amazon referenced 
          links = [a.get('href') for a in soup.find_all('a', href=True)]
          for link in links:
            if 'amazon' in link:
              if(link not in amazonLinks):
                amazonLinks.add(link)
                # print(link)
     
          visible_texts = filter(visible, data)
          temp=''
          for t in visible_texts:
            junkLine=False
            for w in junkWords: 
              if w in t:
                junkLine=True
                break
            if not junkLine:
              # print("==============================")
              # print(t)
              # print(t.strip())
              # print(type(t))
              temp =temp+ (t.strip()+"\n")
              # print(temp)
              # print("==============================")
          visible_texts=temp
          pages.append(visible_texts)
      except Exception as e:
          #print('couldnt get page',items)
          print(e)
      #soup = BeautifulSoup(html)
      #data = soup.findAll(text=True)
      print(cc)
      cc+=1
  return pages,amazonLinks

  #how many pages have visible text to scrape 
  # print(len(pages))
#all pages with visible text from query
def findLists(query,linkcount):
  pages,amazonLinks=getPages(getLinks(query,linkcount))
  unsearchedPages=[]
  lists=[]

  regEx = "\d+[.][' '][a-zA-z].*"

  for page in pages:
    #come back and do something with page if no rankings are found
    items=re.findall(regEx,page)
    if len(items)==0:
      unsearchedPages.append(page)

    else:
      lists.append(items)
  # print("=================================== PRODUCT NAMES ===============================")
  # for list in lists:
  #   print(list)
  # print("=================================== PRODUCT NAMES ===============================")
  return lists,amazonLinks,unsearchedPages





In [0]:
 def listformat(lstoflst):
  #this section turns each item into a list [rank,name], so we have [[[rank, name][rank,name]][[rank,name][rank,name]]]
  newlist=[]
  for i in range(len(lstoflst)):
    templist=[]
    print("============================= PAGE",i+1,"=============================")
    for k in range(len(lstoflst[i])):
      print(lstoflst[i][k])
      rank=lstoflst[i][k][0:3]
      name=lstoflst[i][k][3:]
      templist.append([rank,name])
    newlist.append(templist)
  return(newlist)

      



def listcleanblanks(lst):
  for i in (lst[:]):
    if i==[]:
      lst.remove(i)
  return(lst)

def removespaceandcombo(lst):
  for i in range(len(lst)):
    for k in range(len(lst[i])):
      
      if lst[i][k][1][0] == ' ':
        lst[i][k][1]=lst[i][k][1][1:]
      if lst[i][k][0][-1]==' ':
        lst[i][k][0]=lst[i][k][0][:-1]
  newlist=[]
  for i in range(len(lst)):
    for k in range(len(lst[i])):
      newlist.append(lst[i][k])

  return(newlist)

def getAmazonASIN(amazonLinks):
  productsASIN=set()
  urls=set()
  for link in amazonLinks:
      try:
        headers = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, sdch, br",
        "Accept-Language": "en-US,en;q=0.8",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36"}
        regEx= '[A-Z0-9]{10}[?]'
        request = requests.get(link, headers=headers, allow_redirects=True)
        url = request.url
        ASIN = re.search(regEx,url)
        if(ASIN!=None):
          urls.add(url)
          ASIN = ASIN.group(0)        
          ASIN=ASIN[:10]
          productsASIN.add(ASIN)
      except Exception as e:
        print(e) 
  return productsASIN    


# lst,amazonLinks,unsearchedPages=findLists()
# lst=listformat(lst)
# lst=listcleanblanks(lst)
# #lst=removespaceandcombo(lst)
# amazonASIN= getAmazonASIN(amazonLinks)
# print(*lst,sep='/n')
# print('-----------------------------')
# numoflists=len(lst)



In [0]:
def createASINPairs(ASIN):
  trueNames=dict()
  wd = webdriver.Chrome('chromedriver',options=options)
  for id in ASIN:
    try:
      query = 'http://www.amazon.com/s?k=ASIN&ref=nb_sb_noss'
      query= query.replace('ASIN',str(id))
      wd.get(query)
      trueNames[wd.find_element_by_class_name('s-image').get_attribute("alt")]=id
    except Exception as e:
      #print(e,query)
  for k, v in trueNames.items():
    #print (k, '-->', v)
  return trueNames
# ooga=createASINPairs(amazonASIN)



In [0]:
def similar(product, url):
  Token_Set_Ratio = fuzz.token_set_ratio(product.lower(),url.lower())
  #print(Token_Set_Ratio)
  return Token_Set_Ratio

def compareNames(lst,nameDict):
  rankings=[]
  for site in lst:
    for i in range(len(site)):
      for name in nameDict:
        if similar(site[i][1],name) >= 80:
          rankings.append([site[i][1],similar(site[i][1],name),name])
 # print(*rankings,sep='\n')
  rankings = sorted(rankings, key = lambda x: (x[0], x[1]))
  rankings.reverse()
  print(*rankings,sep='\n')
  newlist=[]
  for site in lst:
    for i in range(len(site)):
      for k in range(len(rankings)):
        if site[i][1] == rankings[k][0]:
          newlist.append([site[i][0],rankings[k][0],rankings[k][2]])
          break;
  #print(*newlist,sep='\n')
  return newlist
def connectasin(lst,namedict):
  newlist=[]
  for item in lst:
    newlist.append([item[0],item[2],namedict[item[2]]])
  #print(newlist)
  return newlist
# one=compareNames(lst,ooga)
# two=connectasin(one,ooga)  
# print(*two,sep='\n')
# # lst=compareNames(lst,ooga)
# # lst=connectasin(lst,ooga)

In [0]:
#recieving list of lists, each element of the form ['rank from a site', 'name', 'ASIN']
def organizetriplelist(lst):
  for i in range(len(lst)):
    loc=lst[i][0].find('.')
    lst[i][0]=lst[i][0][0:loc]
  for i in range(len(lst)):
    lst[i].reverse()
    lst[i][-1]=int(lst[i][-1])
    
  return lst



def listcompression(lst):
  asin=[]
  newlist=[]
  for i in range(len(lst)):
    asin.append(lst[i][0])
  asin=set(asin)
  for i in asin:
     newlist.append([i])
  for i in range(len(newlist)):
    for k in range(len(lst)):
      if lst[k][0]==newlist[i][0]:
        if len(newlist[i])==1:
          newlist[i].append(lst[k][1])
        newlist[i].append(lst[k][2])

  for i in range(len(newlist)):
    newlist[i].append((sum(newlist[i][2:]))/(len(newlist[i])-2))
    newlist[i].append(len(newlist[i])-3)
  return(newlist)

def addnumoflists(lst,num):
  for i in range(len(lst)):
    lst[i].append(num)
  return lst


 #five = [asin,name,...,ave,freq,#oflists]
def addscore(lst):
  for i in range(len(lst)):
    score = lst[i][-3] + -20*(math.log((2/3)+((lst[i][-2])/(lst[i][-1]))))
    lst[i].append(score)
  return lst


def finalsortbyscore(lst):
  lst.sort(key = lambda lst: lst[-1])
  return lst
#all the number variable names are troys responsibility because nobody knows what any of that means 
# print('===========================================')
# three=organizetriplelist(two)
# # print(*three,sep='\n')
# # print('===========================================')
# four=listcompression(three)
# # print(*four,sep='\n')
# # print('===========================================')
# five=addnumoflists(four,numoflists)
# # print(*five,sep='\n')
# # print('===========================================')
# six=addscore(five)
# #print(*six,sep='\n')

# seven = finalsortbyscore(six)
# print('===========================================')
# print(*seven,sep='\n')

# demolist=[]
# demolist.append(seven)



# lst=testing()
# print('========================')
# lst=listcompression(lst)
# print(*lst,sep='\n')

In [0]:
def oogabooga(query,linkcount):
  one,amazonLinks,unsearched=findLists(query,linkcount)
  two=listformat(one)
  three=listcleanblanks(two)
  numoflists=len(three)
  #lst=removespaceandcombo(lst)
  amazonASIN= getAmazonASIN(amazonLinks)
  ooga=createASINPairs(amazonASIN)
  four=compareNames(three,ooga)
  five=connectasin(four,ooga)

  six=organizetriplelist(five)
  seven=listcompression(six)
  eight=addnumoflists(seven,numoflists)
  nine=addscore(eight)
  ten = finalsortbyscore(nine)
  print(*ten,sep='\n')
  return ten
  

#oogabooga('top 10 gaming headphones',20)

In [0]:
qlist=['top 10 gaming mice', 'top 10 hammers','top 10 gaming keyboards','top 10 microphones under $200','top 10 amazon deals']
demolist=[]
for i in qlist:
  demolist.append(oogabooga(i,30))


0
1
7
11
12
13
17
18
19
20
21
22
23
24
25
26
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
============================= PAGE 1 =============================
1. SteelSeries Rival 710
2. HyperX Pulsefire Surge
3. Razer Viper
4. Corsair Harpoon RGB Wireless
5. SteelSeries Sensei Ten
6. Razer Basilisk X Hyperspeed
7. Corsair IronClaw RGB
8. Roccat Kain 100 Aimo
9. Logitech G502 Hero
10. Roccat Kain 120 AIMO
10. Gigabyte Aorus M5
============================= PAGE 2 =============================
1. Logitech G502 Lightspeed Wireless
2. Razer Deathadder Elite
3. Logitech G203 Prodigy
4. Corsair Ironclaw RGB
5. Razer Naga Trinity
6. Steelseries Sensei 310
7. Razer Viper Ultimate Wireless
8. Logitech MX Vertical
9. Logitech G Pro Wireless
10. Logitech G604 Lightspeed
============================= PAGE 3 =============================
1. Roccat Kain 120 AIMO
2. Logitech G502 Lightspeed
3. Asus ROG Spatha
4. Razer Deathadder Elite
5. Roccat Kone Pure Ultra
6. HyperX Pulsefire Surge
7. Corsair Sabre RGB
